In [1]:
import pandas as pd
import numpy as np
data=pd.read_excel('cust_2.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   fashion_prod  35 non-null     float64
 1   Savings       35 non-null     float64
dtypes: float64(2)
memory usage: 692.0 bytes


In [3]:
import statsmodels.api as sm
X=sm.add_constant(data['fashion_prod'])
X.head(5)

,const,fashion_prod
0,1.0,9.10
1,1.0,0.23
2,1.0,1.92
3,1.0,18.20
4,1.0,0.10


In [6]:
Y=data['Savings']

In [7]:

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split( X,Y, train_size = 0.8, random_state = 100 )


In [12]:
#fitting the model
data_lm=sm.OLS(train_y,train_X).fit()
print(data_lm.params)

const          -8.720007
fashion_prod    1.057888
dtype: float64


In [21]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
model=LinearRegression()
model.fit(train_X, train_y)
y_pred = model.predict(test_X)

r_squared = r2_score(test_y, y_pred)
print(f'R-squared: {r_squared:.2f}')

R-squared: 0.41


In [22]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test_y, y_pred)

print(f'Mean Squared Error (MSE): {mse:.2f}')


Mean Squared Error (MSE): 12.81


In [24]:
from scipy.stats import zscore
data['z_score_y']=zscore(data.Savings)
data[ (data.z_score_y > 3.0) | (data.z_score__y < -3.0) ]

AttributeError: 'DataFrame' object has no attribute 'z_score__y'

In [26]:
from scipy import stats
from scipy.stats import zscore
# Calculate the Z-scores for the residuals
residuals = test_y - y_pred
z_scores = np.abs(stats.zscore(residuals))
data['z_scores']=zscore(data.Savings)
data[ (data.z_scores > 3.0) | (data.z_scores < -3.0) ]

print("Outliers detected by Z-score method:")
print(z_scores)

Outliers detected by Z-score method:
33    1.447168
34    0.551959
31    0.367836
5     0.112977
1     0.476831
13    2.018068
18    0.391168
Name: Savings, dtype: float64


In [1]:
from statsmodels.stats.outliers_influence import OLSInfluence



# Calculate Cook's distance
influence = OLSInfluence(data_lm)
cook_distance = influence.cooks_distance[0]

# Set a threshold for Cook's distance (e.g., 1)
cook_threshold = 1

# Find the outliers based on Cook's distance
outliers_cook = np.where(cook_distance > cook_threshold)
data['outliers_cook']=data.Savings
print("Outliers detected by Cook's distance method:")
print(outliers_cook)
#outliers_cook_values = test_y.iloc[outliers_cook]
#print(outliers_cook_values)


NameError: name 'data_lm' is not defined

In [33]:
# Calculate the leverage values
leverage = influence.hat_matrix_diag

# Set a threshold for leverage (e.g., 2 times the mean leverage)
#leverage_threshold = 2 * np.mean(leverage)
data['leverage']=data.Savings
data[ (data.leverage > 3.0) | (data.leverage < -3.0) ]
# Find the outliers based on leverage
#outliers_leverage = np.where(leverage > leverage_threshold)

print("Outliers detected by Leverage method:")
print(leverage)


Outliers detected by Leverage method:
[0.05311796 0.04716051 0.07960514 0.03635198 0.06254    0.03708975
 0.08909073 0.20246862 0.039995   0.08077373 0.14512782 0.13615503
 0.05842004 0.05461072 0.04376572 0.05385666 0.06776395 0.07731402
 0.06190845 0.0784519  0.05311796 0.04965473 0.03595625 0.04338672
 0.03803462 0.11936092 0.05368646 0.1012346 ]
